In [1]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [3]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
#time.sleep(2) # 파이썬 파일에서 이 작업을 안하면 에러 발생

In [4]:
#trs = driver.find_elements_by_tag_name('tr')

trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [7]:
#soup.lelect('from > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [8]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip('\n[]')
category

'음악/댄스/가수]\n                                \n                                '

In [9]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [10]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]

subscriber, view, video


('6400만', '190억0381만', '371')

In [11]:
channels = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category, name, subscriber, view, video])


In [12]:
df = pd.DataFrame(channels, columns = ['카테고리', '채널명','구독자수','조회수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [13]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6557만,433
97,음악/댄스/가수,NCT DREAM,386만,3억7422만,228
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"
99,취미/라이프,JaeYeol ASMR 재열,380만,10억1154만,970


- 만과 억을 숫자로 바꿔주는 함수

In [15]:
def convert_unit_2(s):
    #억 은 지우고 만은 0000을붙인다.
    s = ''.join(s.split('억'))
    s=s.replace('만','0000')
    return s

In [17]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [18]:

convert_unit('10억6557만')

'1,065,570,000'

In [ ]:
#두번쨰 페이지

In [37]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[2]').click()

In [39]:
html = driver.page_source
soup  = BeautifulSoup(html, 'html.parser')

In [40]:
channel_list= soup.select('.aos-init')
len(channel_list)

100

In [41]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = channel.select_one('.subscriber_cnt').text
    view = channel.select_one('.view_cnt').text
    video = channel.select_one('.video_cnt').text[:-1]
    channels.append([category, name, subscriber, view, video])

In [42]:
df = pd.DataFrame(channels, columns = ['카테고리', '채널명','구독자수','조회수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [19]:
## page 1에서 10까지 크롤링하기

In [20]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3) # 중요, 충분한 로딩 시간 확보, 안할시 에러 발생
    html = driver.page_source
    soup  = BeautifulSoup(html, 'html.parser')
    channel_list= soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = channel.select_one('.subscriber_cnt').text
        view = channel.select_one('.view_cnt').text
        video = channel.select_one('.video_cnt').text[:-1]
        results.append([category, name, subscriber, view, video])

In [13]:
df = pd.DataFrame(results, columns = ['카테고리', '채널명','구독자수','조회수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223


In [14]:
df.tail()


,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,51만,2억1077만,"3,084"
996,미분류,밉지않은 관종언니,51만,6774만,170
997,미분류,복지마블TV [Welfare Marble],51만,4996만,240
998,음식/요리/레시피,Muggo,51만,1억8372만,"1,388"
999,게임,임선비,51만,2억3429만,"1,134"


In [15]:
df.to_csv('유튜브_순위.csv')

In [16]:
driver.close()